In [27]:
import json
import pandas as pd
import requests
import csv
import seaborn as sns

In [ ]:
# We doen een "aanvraag" bij de API. Deze url werkt alleen als de rest server gestart is. 
response = requests.get('http://localhost:8080/medish_centrum_randstad/api/netlify?page=1')

In [29]:
# Checken wat voor data/type we binnen hebben gekregen
# response.headers['content-type']

In [41]:
#Nieuwe data opgeslagen

df = pd.read_csv('../../Projects/Project1/data/data.csv')
df.head()

,genetic,length,mass,exercise,smoking,alcohol,lifespan,sugar
0,73.9,185.0,99.7,0.9,0.0,2.4,73.1,6.9
1,86.0,172.0,105.4,1.8,8.1,0.4,85.0,4.2
2,83.3,176.0,111.4,1.1,0.8,4.6,81.6,7.5
3,82.8,164.0,111.4,4.7,11.8,1.0,81.0,2.9
4,78.7,178.0,71.6,1.5,8.3,4.9,75.0,5.5


In [ ]:
#verwijder column 'id'

#df.drop('id', axis=1, inplace=True)
# df =df.drop(df.columns[0], axis = 1)

# df.head()

In [3]:
# We hebben nu een werkbaar dataframe met al onze variabelen in de kolommen, en alle individuele patienten/datapunten
# in de rijen. We kunnen deze nu evt. als een nieuwe csv file exporteren, zonder index:

df.to_csv('data1.csv', index=False, sep=',')


In [31]:
# Als een van de eerste stappen in de IDE kunnen we kijken naar wat voor soort variabelen we hebben,
# van welk type deze variabelen zijn (categorisch of numeriek), en ook of er bijvoorbeeld waardes missen. 

# Eerst kijken we naar de datatypen van elke kolom
print(type(df))
print(df.dtypes)
print(df.info())
print(df)

<class 'pandas.core.frame.DataFrame'>
genetic     float64
length      float64
mass        float64
exercise    float64
smoking     float64
alcohol     float64
lifespan    float64
sugar       float64
dtype: object
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4096 entries, 0 to 4095
Data columns (total 8 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   genetic   4095 non-null   float64
 1   length    4093 non-null   float64
 2   mass      4094 non-null   float64
 3   exercise  4095 non-null   float64
 4   smoking   4096 non-null   float64
 5   alcohol   4094 non-null   float64
 6   lifespan  4094 non-null   float64
 7   sugar     4095 non-null   float64
dtypes: float64(8)
memory usage: 256.1 KB
None
      genetic  length   mass  exercise  smoking  alcohol  lifespan  sugar
0        73.9   185.0   99.7       0.9      0.0      2.4      73.1    6.9
1        86.0   172.0  105.4       1.8      8.1      0.4      85.0    4.2
2        83.3   176.0  

In [34]:
#Zoek en verwijder non-numerieke waardes

for c in df.columns:
    df[c] = pd.to_numeric(df[c], errors='coerce')

print(df)

      genetic  length   mass  exercise  smoking  alcohol  lifespan  sugar  \
0        73.9   185.0   99.7       0.9      0.0      2.4      73.1    6.9   
1        86.0   172.0  105.4       1.8      8.1      0.4      85.0    4.2   
2        83.3   176.0  111.4       1.1      0.8      4.6      81.6    7.5   
3        82.8   164.0  111.4       4.7     11.8      1.0      81.0    2.9   
4        78.7   178.0   71.6       1.5      8.3      4.9      75.0    5.5   
...       ...     ...    ...       ...      ...      ...       ...    ...   
4091     80.3   176.0  115.9       3.7      0.1      5.6      80.7    5.8   
4092     75.3   162.0  129.6       3.6      6.2      5.6      71.1    7.6   
4093     93.1   189.0  115.3       2.7      9.4      4.9      90.0    6.6   
4094     77.9   170.0   90.8       2.3     13.2      0.8      75.9    6.2   
4095    101.3   188.0  129.2       1.2      8.2      6.0      96.9    6.2   

       BMI  
0     29.1  
1     35.6  
2     36.0  
3     41.4  
4     22.6

In [35]:
#vind alle null-waardes en verwijder ze

df= df.dropna(axis=0)
print(df)

      genetic  length   mass  exercise  smoking  alcohol  lifespan  sugar  \
0        73.9   185.0   99.7       0.9      0.0      2.4      73.1    6.9   
1        86.0   172.0  105.4       1.8      8.1      0.4      85.0    4.2   
2        83.3   176.0  111.4       1.1      0.8      4.6      81.6    7.5   
3        82.8   164.0  111.4       4.7     11.8      1.0      81.0    2.9   
4        78.7   178.0   71.6       1.5      8.3      4.9      75.0    5.5   
...       ...     ...    ...       ...      ...      ...       ...    ...   
4091     80.3   176.0  115.9       3.7      0.1      5.6      80.7    5.8   
4092     75.3   162.0  129.6       3.6      6.2      5.6      71.1    7.6   
4093     93.1   189.0  115.3       2.7      9.4      4.9      90.0    6.6   
4094     77.9   170.0   90.8       2.3     13.2      0.8      75.9    6.2   
4095    101.3   188.0  129.2       1.2      8.2      6.0      96.9    6.2   

       BMI  
0     29.1  
1     35.6  
2     36.0  
3     41.4  
4     22.6

In [36]:
df_null = df[df.isna().any(axis=1)]

df_null

,genetic,length,mass,exercise,smoking,alcohol,lifespan,sugar,BMI


In [38]:
#Er zijn geen null-waardes (meer). 
#Verwijder eventuele negatieve waardes

df.drop(df[df['genetic'] < 0].index, inplace = True)
df.drop(df[df['length'] < 0].index, inplace = True)
df.drop(df[df['mass'] < 0].index, inplace = True)
df.drop(df[df['exercise'] < 0].index, inplace = True)
df.drop(df[df['smoking'] < 0].index, inplace = True)
df.drop(df[df['alcohol'] < 0].index, inplace = True)
df.drop(df[df['lifespan'] < 0].index, inplace = True)
df.drop(df[df['sugar'] < 0].index, inplace = True)
# df.drop(df[df['BMI'] < 0].index, inplace = True)

df



C:\Users\noeki\AppData\Local\Temp\ipykernel_12980\4001715979.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.drop(df[df['genetic'] < 0].index, inplace = True)
C:\Users\noeki\AppData\Local\Temp\ipykernel_12980\4001715979.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.drop(df[df['length'] < 0].index, inplace = True)
C:\Users\noeki\AppData\Local\Temp\ipykernel_12980\4001715979.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-co

,genetic,length,mass,exercise,smoking,alcohol,lifespan,sugar,BMI
0,73.9,185.0,99.7,0.9,0.0,2.4,73.1,6.9,29.1
1,86.0,172.0,105.4,1.8,8.1,0.4,85.0,4.2,35.6
2,83.3,176.0,111.4,1.1,0.8,4.6,81.6,7.5,36.0
3,82.8,164.0,111.4,4.7,11.8,1.0,81.0,2.9,41.4
4,78.7,178.0,71.6,1.5,8.3,4.9,75.0,5.5,22.6
...,...,...,...,...,...,...,...,...,...
4091,80.3,176.0,115.9,3.7,0.1,5.6,80.7,5.8,37.4
4092,75.3,162.0,129.6,3.6,6.2,5.6,71.1,7.6,49.4
4093,93.1,189.0,115.3,2.7,9.4,4.9,90.0,6.6,32.3
4094,77.9,170.0,90.8,2.3,13.2,0.8,75.9,6.2,31.4


In [43]:
#sla de opgeschoonde dataset op 
df.to_csv("dataBMI1.csv")

df

,genetic,length,mass,exercise,smoking,alcohol,lifespan,sugar,BMI
0,73.9,185.0,99.7,0.9,0.0,2.4,73.1,6.9,29.1
1,86.0,172.0,105.4,1.8,8.1,0.4,85.0,4.2,35.6
2,83.3,176.0,111.4,1.1,0.8,4.6,81.6,7.5,36.0
3,82.8,164.0,111.4,4.7,11.8,1.0,81.0,2.9,41.4
4,78.7,178.0,71.6,1.5,8.3,4.9,75.0,5.5,22.6
...,...,...,...,...,...,...,...,...,...
4091,80.3,176.0,115.9,3.7,0.1,5.6,80.7,5.8,37.4
4092,75.3,162.0,129.6,3.6,6.2,5.6,71.1,7.6,49.4
4093,93.1,189.0,115.3,2.7,9.4,4.9,90.0,6.6,32.3
4094,77.9,170.0,90.8,2.3,13.2,0.8,75.9,6.2,31.4


In [ ]:
print(df.nunique())

In [44]:
# Een volgende stap is om eens te kijken naar wat van de variabelen, hoe zit het bijvoorbeeld met de gemiddelde
# levensverwachting en suikerinname? 

df[['sugar', 'lifespan']].groupby(['sugar'], as_index=False).mean().sort_values(by='sugar', ascending=False)

,sugar,lifespan
131,13.8,77.985714
130,13.7,74.950000
129,13.6,77.575000
128,13.5,79.533333
127,13.4,82.733333
...,...,...
4,1.1,79.157143
3,1.0,80.920000
2,0.9,85.357143
1,0.8,80.030000


In [ ]:
# Hier zien we een duidelijke afname in levensverwachting naarmate 'sugar' een hogere waarde aanneemt. 
# Hetzelfde kunnen we doen voor 'exercise'

df[['exercise', 'lifespan']].groupby(['exercise'], as_index=False).mean().sort_values(by='lifespan', ascending=False)

In [ ]:
# Wat niet zozeer interessant is voor ons toekomstige model, maar wel interessant: hoe is het gesteld met het bewegen
# in deze dataset; 

#df['exercise'].value_counts()

In [ ]:
#We kunnen nu kijken naar eventuele onderlinge correlaties:
print(df.corr())
sns.heatmap(df.corr())

In [ ]:
df.corr()['lifespan']

In [ ]:
#controleer op outliers met boxplots

sns.boxplot(x=df['genetic'])

#Alleen bij 'genetic' en 'lifespan' zijn er outliers boven de 100 jaar


In [49]:
# #voeg column BMI toe
df = pd.read_csv('../../Projects/Project1/data/data.csv')
df["BMI"] = (df.mass/(df.length/100)**2)

#df

In [50]:
#round BMI value off to 1 decimal. 

df['BMI'] = df['BMI'].apply(lambda x: round(x, 1)) 

print(df)


      genetic  length   mass  exercise  smoking  alcohol  lifespan  sugar  \
0        73.9   185.0   99.7       0.9      0.0      2.4      73.1    6.9   
1        86.0   172.0  105.4       1.8      8.1      0.4      85.0    4.2   
2        83.3   176.0  111.4       1.1      0.8      4.6      81.6    7.5   
3        82.8   164.0  111.4       4.7     11.8      1.0      81.0    2.9   
4        78.7   178.0   71.6       1.5      8.3      4.9      75.0    5.5   
...       ...     ...    ...       ...      ...      ...       ...    ...   
4091     80.3   176.0  115.9       3.7      0.1      5.6      80.7    5.8   
4092     75.3   162.0  129.6       3.6      6.2      5.6      71.1    7.6   
4093     93.1   189.0  115.3       2.7      9.4      4.9      90.0    6.6   
4094     77.9   170.0   90.8       2.3     13.2      0.8      75.9    6.2   
4095    101.3   188.0  129.2       1.2      8.2      6.0      96.9    6.2   

       BMI  
0     29.1  
1     35.6  
2     36.0  
3     41.4  
4     22.6

In [ ]:
#drop columns mass en length

# df_bmi = df.drop(['length', 'mass'], axis=1)
# df_bmi

In [20]:
df_lifestyle = df[['exercise', 'smoking', 'alcohol', 'sugar', 'lifespan', 'BMI']]

df_not_genetic = df.drop(['genetic'], axis = 1)
df_ng = df_not_genetic

In [22]:
# save df

# df.to_csv("data_clean1.csv")

In [51]:
# df met BMI, maar zonder mass+length

df_bmi = df.drop(['length', 'mass'], axis=1)

df_bmi

,genetic,exercise,smoking,alcohol,lifespan,sugar,BMI
0,73.9,0.9,0.0,2.4,73.1,6.9,29.1
1,86.0,1.8,8.1,0.4,85.0,4.2,35.6
2,83.3,1.1,0.8,4.6,81.6,7.5,36.0
3,82.8,4.7,11.8,1.0,81.0,2.9,41.4
4,78.7,1.5,8.3,4.9,75.0,5.5,22.6
...,...,...,...,...,...,...,...
4091,80.3,3.7,0.1,5.6,80.7,5.8,37.4
4092,75.3,3.6,6.2,5.6,71.1,7.6,49.4
4093,93.1,2.7,9.4,4.9,90.0,6.6,32.3
4094,77.9,2.3,13.2,0.8,75.9,6.2,31.4


In [24]:
# df_bmi.to_csv("DataBMI.csv")